<a href="https://colab.research.google.com/github/phonhay103/anything/blob/main/MobileNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
import tensorflow as tf

In [21]:
def conv_block(inputs, filters, alpha, kernel=(3, 3), strides=(1, 1)):
    filters = int(alpha * filters)
    x = tf.keras.layers.Conv2D(
        filters,
        kernel,
        padding='same',
        use_bias=False,
        strides=strides,
        name='conv1'
    )(inputs)
    x = tf.keras.layers.BatchNormalization(name='conv1_bn')(x)
    return tf.keras.layers.ReLU(6., name='conv1_relu')(x)

def depthwise_conv_block(inputs, pointwise_conv_filters, alpha, depth_multiplier=1, strides=(1, 1), block_id=1):
    pointwise_conv_filters = int(alpha * pointwise_conv_filters)
    if strides == (1, 1):
        x = inputs
    else:
        x = tf.keras.layers.ZeroPadding2D(((0, 1), (1, 0)), name='conv_pad_%d' % block_id)(inputs)
    x = tf.keras.layers.DepthwiseConv2D((3, 3),
                                        padding='same' if strides == (1, 1) else 'valid',
                                        depth_multiplier=depth_multiplier,
                                        strides=strides,
                                        use_bias=False,
                                        name='conv_dw_%d' % block_id)(x)
    x = tf.keras.layers.BatchNormalization(name='conv_dw_%d_bn' % block_id)(x)
    x = tf.keras.layers.ReLU(6., name='conv_dw_%d_relu' % block_id)(x)

    x = tf.keras.layers.Conv2D(
        pointwise_conv_filters,
        (1, 1),
        padding='same',
        use_bias=False,
        strides=(1, 1),
        name='conv_pw_%d' % block_id)(x)
    x = tf.keras.layers.BatchNormalization(name='conv_pw_%d_bn' % block_id)(x)
    return tf.keras.layers.ReLU(6., name='conv_pw_%d_relu' % block_id)(x)

def MobileNet(input_shape=None, alpha=1.0, depth_multiplier=1,  dropout=1e-3, classes=1000):
    # Check
    if alpha not in [0.25, 0.50, 0.75, 1.0]:
        raise ValueError('If imagenet weights are being loaded, alpha can be one of `0.25`, `0.50`, `0.75` or `1.0` only.')

    # Input Shape
    if input_shape is None:
        input_shape = (224, 224, 3)
    else:
        rows = input_shape[0]
        cols = input_shape[1]
        if rows == cols and rows in [128, 160, 192, 224]:
            input_shape = (rows, rows, 3)
        else:
            input_shape = (224, 224, 3)
    
    # Feature Extraction
    img_input = tf.keras.layers.Input(shape=input_shape, name='input')
    x = conv_block(img_input, 32, alpha, strides=(2, 2))
    x = depthwise_conv_block(x, 64, alpha, depth_multiplier, block_id=1)
    x = depthwise_conv_block(x, 128, alpha, depth_multiplier, strides=(2, 2), block_id=2)
    x = depthwise_conv_block(x, 128, alpha, depth_multiplier, block_id=3)
    x = depthwise_conv_block(x, 256, alpha, depth_multiplier, strides=(2, 2), block_id=4)
    x = depthwise_conv_block(x, 256, alpha, depth_multiplier, block_id=5)
    x = depthwise_conv_block(x, 512, alpha, depth_multiplier, strides=(2, 2), block_id=6)
    x = depthwise_conv_block(x, 512, alpha, depth_multiplier, block_id=7)
    x = depthwise_conv_block(x, 512, alpha, depth_multiplier, block_id=8)
    x = depthwise_conv_block(x, 512, alpha, depth_multiplier, block_id=9)
    x = depthwise_conv_block(x, 512, alpha, depth_multiplier, block_id=10)
    x = depthwise_conv_block(x, 512, alpha, depth_multiplier, block_id=11)
    x = depthwise_conv_block(x, 1024, alpha, depth_multiplier, strides=(2, 2), block_id=12)
    x = depthwise_conv_block(x, 1024, alpha, depth_multiplier, block_id=13)

    # Classification
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Reshape((1, 1, int(1024 * alpha)), name='reshape_1')(x)
    x = tf.keras.layers.Dropout(dropout, name='dropout')(x)
    x = tf.keras.layers.Conv2D(classes, (1, 1), padding='same', name='conv_preds')(x)
    x = tf.keras.layers.Reshape((classes,), name='reshape_2')(x)
    
    # Model
    return tf.keras.Model(inputs=img_input, outputs=x)

In [23]:
model = MobileNet()

In [25]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['acc'])